# Module 19: Prompt Engineering

**The Art of Communicating with LLMs**

---

## 1. Objectives

- ✅ Master prompting techniques
- ✅ Implement few-shot learning
- ✅ Use Chain-of-Thought prompting
- ✅ Build reusable prompt templates

## 2. Prerequisites

- [Module 18: Large Language Models](../18_llms/18_llms.ipynb)

## 3. Prompt Components

```
┌─────────────────────────────────────┐
│ SYSTEM: Role and behavior          │
├─────────────────────────────────────┤
│ CONTEXT: Background information    │
├─────────────────────────────────────┤
│ EXAMPLES: Few-shot demonstrations  │
├─────────────────────────────────────┤
│ INSTRUCTION: What to do            │
├─────────────────────────────────────┤
│ INPUT: User's actual input         │
├─────────────────────────────────────┤
│ OUTPUT FORMAT: How to respond      │
└─────────────────────────────────────┘
```

In [1]:
from transformers import pipeline

# Using GPT-2 for demos (replace with better model in practice)
generator = pipeline('text-generation', model='gpt2', max_new_tokens=50)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


## 4. Zero-Shot Prompting

In [2]:
# Zero-shot: No examples
zero_shot_prompt = """
Classify the sentiment of this review as positive or negative.

Review: The movie was absolutely fantastic!
Sentiment:"""

print("Zero-shot prompt:")
print(zero_shot_prompt)

Zero-shot prompt:

Classify the sentiment of this review as positive or negative.

Review: The movie was absolutely fantastic!
Sentiment:


## 5. Few-Shot Prompting

In [3]:
# Few-shot: Include examples
few_shot_prompt = """
Classify the sentiment of reviews.

Review: Great product, highly recommend!
Sentiment: positive

Review: Terrible quality, waste of money.
Sentiment: negative

Review: It works okay, nothing special.
Sentiment: neutral

Review: The movie was absolutely fantastic!
Sentiment:"""

print("Few-shot prompt:")
print(few_shot_prompt)

Few-shot prompt:

Classify the sentiment of reviews.

Review: Great product, highly recommend!
Sentiment: positive

Review: Terrible quality, waste of money.
Sentiment: negative

Review: It works okay, nothing special.
Sentiment: neutral

Review: The movie was absolutely fantastic!
Sentiment:


## 6. Chain-of-Thought (CoT)

### Key Insight
Make the model "think step by step" for reasoning tasks.

In [4]:
# Without CoT
simple_prompt = """
Q: If a bag has 3 apples and 2 oranges, and you remove 1 apple,
   how many fruits remain?
A:"""

# With CoT
cot_prompt = """
Q: If a bag has 3 apples and 2 oranges, and you remove 1 apple,
   how many fruits remain?

Let's think step by step:
1. Initial apples: 3
2. Initial oranges: 2
3. Total initial fruits: 3 + 2 = 5
4. After removing 1 apple: 5 - 1 = 4

A: 4 fruits remain.

Q: A store has 15 books. They sell 8 and receive 5 new ones.
   How many books do they have?

Let's think step by step:"""

print("Chain-of-Thought prompt:")
print(cot_prompt)

Chain-of-Thought prompt:

Q: If a bag has 3 apples and 2 oranges, and you remove 1 apple, 
   how many fruits remain?

Let's think step by step:
1. Initial apples: 3
2. Initial oranges: 2
3. Total initial fruits: 3 + 2 = 5
4. After removing 1 apple: 5 - 1 = 4

A: 4 fruits remain.

Q: A store has 15 books. They sell 8 and receive 5 new ones. 
   How many books do they have?

Let's think step by step:


## 7. Prompt Templates

In [5]:
class PromptTemplate:
    """Reusable prompt template."""

    def __init__(self, template: str):
        self.template = template

    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

# Classification template
classification_template = PromptTemplate("""
You are a {domain} expert. Classify the following text.

Categories: {categories}
Text: {text}

Classification:""")

# Use it
prompt = classification_template.format(
    domain="sentiment analysis",
    categories="positive, negative, neutral",
    text="I really enjoyed this product!"
)
print(prompt)


You are a sentiment analysis expert. Classify the following text.

Categories: positive, negative, neutral
Text: I really enjoyed this product!

Classification:


In [6]:
# Extraction template
extraction_template = PromptTemplate("""
Extract the following information from the text:
{fields}

Text: {text}

Output as JSON:""")

prompt = extraction_template.format(
    fields="- Name\n- Company\n- Role",
    text="John Smith is the CTO of TechCorp."
)
print(prompt)


Extract the following information from the text:
- Name
- Company
- Role

Text: John Smith is the CTO of TechCorp.

Output as JSON:


## 8. Advanced Techniques

### Role Prompting
```
You are a senior Python developer with 10 years of experience.
Review this code for best practices...
```

### Output Format Control
```
Respond ONLY with valid JSON in this format:
{"answer": "...", "confidence": 0.0-1.0}
```

### Self-Consistency
```
Generate 3 different solutions, then pick the most common answer.
```

In [7]:
# Structured output template
structured_template = PromptTemplate("""
Analyze the following text and respond in JSON format.

Text: {text}

Respond ONLY with valid JSON:
{{
    "main_topic": "...",
    "sentiment": "positive/negative/neutral",
    "key_entities": [...],
    "summary": "..."
}}""")

prompt = structured_template.format(
    text="Apple announced new iPhone today in California. Stock rose 5%."
)
print(prompt)


Analyze the following text and respond in JSON format.

Text: Apple announced new iPhone today in California. Stock rose 5%.

Respond ONLY with valid JSON:
{
    "main_topic": "...",
    "sentiment": "positive/negative/neutral",
    "key_entities": [...],
    "summary": "..."
}


## 9. 🔥 Real-World Usage

### Best Practices

| Technique | When to Use |
|-----------|-------------|
| Zero-shot | Simple tasks, large models |
| Few-shot | Format learning, small models |
| CoT | Math, reasoning, logic |
| Role prompting | Domain expertise needed |

### Common Mistakes

1. ❌ Vague instructions → ✅ Be specific
2. ❌ No examples → ✅ Show desired format
3. ❌ Long prompts → ✅ Concise and clear
4. ❌ No constraints → ✅ Specify output format

## 10. Interview Questions

**Q1: What is few-shot prompting?**
<details><summary>Answer</summary>

Including examples in the prompt to show the model the expected format and behavior. Works better than zero-shot for complex tasks.
</details>

**Q2: When to use Chain-of-Thought?**
<details><summary>Answer</summary>

For reasoning tasks: math, logic, multi-step problems. Add "Let's think step by step" to encourage the model to show its reasoning.
</details>

## 11. Summary

- **Zero-shot**: Direct instruction, no examples
- **Few-shot**: Include examples to guide format
- **Chain-of-Thought**: Step-by-step reasoning
- **Templates**: Reusable, parameterized prompts

## 12. References

- [Chain-of-Thought Paper](https://arxiv.org/abs/2201.11903)
- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [OpenAI Best Practices](https://platform.openai.com/docs/guides/prompt-engineering)

---
**Next:** [Module 20: Fine-Tuning LLMs](../20_finetuning/20_finetuning.ipynb)